# Scoring Reviews based on Amazon Fine Food Reviews using Natural Language Processing

**Aadith Sukumar**<br>
**AIML A1**<br>
**21070126003**<br>

Data Source: [Kaggle](https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews?resource=download)

In [31]:
# Importing the libraries for nlp
import pandas as pd
import nltk
nltk.download("all")
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

In [32]:
# load the data
data = pd.read_csv("/content/drive/MyDrive/NLP Datasets/Reviews.csv")
data.head(10)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,5,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,5,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,5,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...


In [33]:
print("Shape of data: ",data.shape)
data.info()

Shape of data:  (568454, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [34]:
# sampling data to get only 10000 rows for faster processing
data = data.sample(n=10000)
data.head(10)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
469186,469187,B000AQFQC6,A1SU720MCEN7ME,Mark R. Harrison,1,1,2,1140652800,"Good product, terrible shipping","We like the noodles we got, but the problem I ..."
173931,173932,B005HUVI40,A1WMWU12C6DL2T,J. Johnson,1,1,5,1296518400,Good coffee,This is one of the better K-cups. I usually d...
162411,162412,B000YSQ9GC,APTF346VYXUJ6,Firepoke,0,1,5,1306627200,Awesome product,This product has exceeded our expectations. I...
73543,73544,B000YQUY2E,AFZLBRCW8VPPU,A. C. Teeple,4,6,5,1206489600,Wish this Flavor Was Easier to Find,Torani's sugar free brown sugar cinnamon is gr...
218900,218901,B0013E7C0S,AKZWJFBLSZISE,David Capps,1,1,5,1330819200,Taste great,"Shipping was very fast, I am a prime member. U..."
309893,309894,B005BPVCVU,A3S15YGZ6W6EV2,K. Varraso,1,1,2,1329955200,"Convenient, but not very tasty","I ate and thought, but I wasn't able to smile ..."
228641,228642,B0058AMY10,A1X5RAIMX4ZSLJ,BadPotato,0,0,5,1331856000,Best chips ever,"The title says it all. Great flavor, hint of s..."
108804,108805,B001FYUVG8,A15SYT7EXWTJX8,ShoppaHaulic,2,2,5,1331078400,Goodbye Rawhides! Hello Peace of Mind - & Dogs...,"Love these! Well, because...we have 3 dogs (2 ..."
462801,462802,B0075JTH7Q,A1RETZE165IHP8,"Judy Quan ""run26_2chick""",5,6,3,1162512000,"Cytomax drink good, cranberry grapefruit flavo...",Cytomax is a good recovery drink that doesn't ...
401331,401332,B000CQG8B2,A2CGNLD2641LU7,"Suzanne M. Jarrell ""tigertattoo""",2,2,4,1167782400,Loveley & Soothing,This tea is very good. The flavor is great an...


In [35]:
print("Shape of data: ",data.shape)
data.info()

Shape of data:  (10000, 10)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 469186 to 353355
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Id                      10000 non-null  int64 
 1   ProductId               10000 non-null  object
 2   UserId                  10000 non-null  object
 3   ProfileName             9999 non-null   object
 4   HelpfulnessNumerator    10000 non-null  int64 
 5   HelpfulnessDenominator  10000 non-null  int64 
 6   Score                   10000 non-null  int64 
 7   Time                    10000 non-null  int64 
 8   Summary                 9997 non-null   object
 9   Text                    10000 non-null  object
dtypes: int64(5), object(5)
memory usage: 859.4+ KB


## Preprocessing

In [36]:
# lemmatization with spacy
nlp = spacy.load('en_core_web_sm')
data['Text'] = data['Text'].apply(lambda x: nlp(x))

In [37]:
# tokenize data from the tuples of "Text" column
data['Text'] = data['Text'].apply(lambda x: [item.text for item in x])

In [38]:
data.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
469186,469187,B000AQFQC6,A1SU720MCEN7ME,Mark R. Harrison,1,1,2,1140652800,"Good product, terrible shipping","[We, like, the, noodles, we, got, ,, but, the,..."
173931,173932,B005HUVI40,A1WMWU12C6DL2T,J. Johnson,1,1,5,1296518400,Good coffee,"[This, is, one, of, the, better, K, -, cups, ...."


In [39]:
# removing stop words
data['Text'] = data['Text'].apply(lambda x: [item for item in x if item not in STOP_WORDS])
data.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
469186,469187,B000AQFQC6,A1SU720MCEN7ME,Mark R. Harrison,1,1,2,1140652800,"Good product, terrible shipping","[We, like, noodles, got, ,, problem, I, took, ..."
173931,173932,B005HUVI40,A1WMWU12C6DL2T,J. Johnson,1,1,5,1296518400,Good coffee,"[This, better, K, -, cups, ., , I, usually, l..."


In [40]:
# remove symbols and special characters
data['Text'] = data['Text'].apply(lambda x: [item for item in x if item.isalpha()])
data.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
469186,469187,B000AQFQC6,A1SU720MCEN7ME,Mark R. Harrison,1,1,2,1140652800,"Good product, terrible shipping","[We, like, noodles, got, problem, I, took, mon..."
173931,173932,B005HUVI40,A1WMWU12C6DL2T,J. Johnson,1,1,5,1296518400,Good coffee,"[This, better, K, cups, I, usually, like, bold..."


In [41]:
# remove urls
data['Text'] = data['Text'].apply(lambda x: [item for item in x if "http" not in item])
data.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
469186,469187,B000AQFQC6,A1SU720MCEN7ME,Mark R. Harrison,1,1,2,1140652800,"Good product, terrible shipping","[We, like, noodles, got, problem, I, took, mon..."
173931,173932,B005HUVI40,A1WMWU12C6DL2T,J. Johnson,1,1,5,1296518400,Good coffee,"[This, better, K, cups, I, usually, like, bold..."


---

## Vectorizing Data

In [48]:
# Vectorizing using CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# run it on every row in list format
data['Text_Processed'] = data['Text'].apply(lambda x: ' '.join(x))
cv = CountVectorizer(max_features=5000, stop_words='english')
X_CV = cv.fit_transform(data['Text_Processed']).toarray()
print("X_CV-Shape: ",X_CV.shape)

X_CV-Shape:  (10000, 5000)


In [50]:
# Vectorizing using TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
X_TF = tfidf.fit_transform(data['Text_Processed']).toarray()
print("X_TF-Shape: ",X_TF.shape)

X_TF-Shape:  (10000, 5000)


In [52]:
# Vectorizing using Word2Vec
from gensim.models import Word2Vec

model = Word2Vec(data['Text'], min_count=1)
print("model-Shape: ",model.wv.vectors.shape)

model-Shape:  (24583, 100)


In [53]:
# Vectorizing using GoogleNews Word2Vec
from gensim.models import KeyedVectors

model_GN = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/GoogleNews-vectors-negative300.bin.gz', binary=True)
print("model_GN-Shape: ",model_GN.vectors.shape)

model_GN-Shape:  (3000000, 300)


---

## ML Algorithms